These data come from a survey conducted by INSEE, Budget des Familles, which are protected by the Statistical Secrecy Committee. To reproduce the analysis, one must request access to this data.

In [67]:
import pandas as pd
date_analysis = "2023-04"
df_inflation_by_household = pd.read_csv(f'BDF/computed_inflation_by_household_{date_analysis}.csv', index_col="IDENT_MEN")

# Household

In [68]:
df_menage = pd.read_csv("BDF/Csv/MENAGE.csv", sep=";", encoding='latin1')

C:\Users\mdaversa\AppData\Local\Temp\ipykernel_13088\1221685487.py:1: DtypeWarning: Columns (21,22,23,48,49,421) have mixed types. Specify dtype option on import or set low_memory=False.
  df_menage = pd.read_csv("BDF/Csv/MENAGE.csv", sep=";", encoding='latin1')


In [69]:
df_menage = df_menage.set_index("IDENT_MEN")

In [70]:
df_dep_men = pd.read_csv("BDF/Csv/DEPMEN.csv", sep=";", encoding='latin1')

C:\Users\mdaversa\AppData\Local\Temp\ipykernel_13088\3018729231.py:1: DtypeWarning: Columns (354,387,390,395,430,452,502,732,737,766,770,774,778,782,850,1325,1501,1629) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dep_men = pd.read_csv("BDF/Csv/DEPMEN.csv", sep=";", encoding='latin1')


In [71]:
df_dep_men = df_dep_men.set_index("IDENT_MEN")

In [72]:
df_menage = pd.merge(left=df_menage, right=df_dep_men, left_index=True, right_index=True)

In [73]:
variables = ["AGEPR", "TUU", "DNIVIE1", "Stalog", "NENFANTS", "COUPLEPR", "Chaufp", "TYPVOIS"]
df_filtered = df_menage.loc[:, variables]

# Cleaning variables

In [74]:
def clean_variable(df, variable, variable_ref, numerical, labels=[],  bins=[]):
    df[f'{variable}_category'] = pd.cut(x=df[variable], bins=bins, labels=labels, right=False) if numerical else df[variable]
    df = pd.concat([df, pd.get_dummies(data=df[f"{variable}_category"])], axis=1)
    df = df.drop([variable, f"{variable}_category", variable_ref], axis=1)
    return df


Age

In [75]:
df_filtered = clean_variable(df=df_filtered, variable="AGEPR", labels=["Moins de 30 ans", "De 30 à 44 ans", "De 45 à 59 ans", "De 60 à 74 ans", "75 ans et plus"], variable_ref = "De 45 à 59 ans", bins = [0, 30, 45, 60, 75, 102], numerical=True)

Type menage

TUUU

In [76]:
df_filtered = clean_variable(df=df_filtered, variable="TUU", variable_ref = "Ville moyenne", numerical=True, bins=[0,3,5,9], labels=["Rural et petite ville", "Ville moyenne", "Grande ville"])

Qintiles niveau de vie

In [77]:
df_filtered = clean_variable(df=df_filtered, variable="DNIVIE1", variable_ref = "Moyen", numerical=True, bins=[0,3,9,11], labels=["Pauvre", "Moyen", "Riche"])

In [78]:
df_filtered = df_filtered.dropna()

In [79]:
df_filtered = clean_variable(df=df_filtered, variable="COUPLEPR", variable_ref = "N'habite pas en couple", numerical=True, bins=[0,3,4], labels=["Habite en couple", "N'habite pas en couple"])

Stalog

In [80]:
df_filtered = clean_variable(df=df_filtered, variable="Stalog", variable_ref = "Locataire", numerical=True, bins=[0,4,7], labels=["Propriétaire", "Locataire"])

In [81]:
df_filtered = clean_variable(df=df_filtered, variable="Chaufp", variable_ref = "Chauffage central ou mixte", numerical=True, bins=[0,4,5,8], labels=["Chauffage central ou mixte", "Chauffage électrique individuel", "Chauffage non-électrique"])

In [82]:
df_filtered = clean_variable(df=df_filtered, variable="TYPVOIS", variable_ref = "Immeubles", numerical=True, labels=["Pavillonnaire", "Immeubles"], bins=[0,3,6])

# Regression


In [83]:
df_filtered = pd.merge(df_filtered, df_inflation_by_household, left_index=True, right_index=True)

In [84]:
df_filtered = df_filtered.dropna()


In [85]:
y = df_filtered['inflation']
df_filtered = df_filtered.drop(columns=['inflation'])
X = df_filtered


In [86]:
import statsmodels.api as sm
X = sm.add_constant(X)

In [87]:
model = sm.OLS(y, X)

In [88]:
results = model.fit()


In [89]:
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:              inflation   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     82.30
Date:                Wed, 14 Jun 2023   Prob (F-statistic):          9.01e-229
Time:                        17:05:12   Log-Likelihood:                -33631.
No. Observations:               15945   AIC:                         6.729e+04
Df Residuals:                   15930   BIC:                         6.741e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     